In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
%matplotlib inline
from matplotlib import pyplot as plt
from torchvision import datasets, transforms

# 加载 CIFAR10 数据集

In [5]:
# 下载
from torchvision import datasets
data_path = './data-unversioned/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

In [ ]:
# 索引指定的数据
print('CIFAR10 Length: ', len(cifar10))
img, label = cifar10[99]
plt.imshow(img)
plt.show()
f'Img Label: {label}', type(cifar10).__mro__

# 对数据进行变换

#### 把img转化成张量

ToTensor变换将数据从0-255缩小到了0.0-1.0, 且维度顺序是 Channel x Height x Width
但是pyplot, imageio等库读取图像都默认维度顺序是 Height x Width x Channel

In [ ]:
# 法一: 索引出img, 用ToTensor()类的传唤来转化.
from torchvision import transforms
to_tensor = transforms.ToTensor()
img_t = to_tensor(img)
img_t.shape

In [14]:
# 法二: 把变换直接作为transform参数传递给CIFAR10模型.
data_path = './data-unversioned/'
tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False, transform=transforms.ToTensor())
# 此时, 访问数据集的元素将返回一个张量而不是PIL image.

Files already downloaded and verified


In [ ]:
img_t, _ = tensor_cifar10[99]
# img_t.min(), img_t.max() -> tensor(0.), tensor(1.)
# 这说明ToTensor变换将数据从0-255缩小到了0.0-1.0
plt.imshow(img_t.permute(1, 2, 0))  # turn it into numpy order
plt.show()

#### 数据归一化

In [ ]:
imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3).cuda()
imgs.device, imgs.shape

In [11]:
imgs.reshape(3, -1).mean(dim=1)     # 保留3个通道, 把32x32像素格缩到一个维度里

tensor([0.4914, 0.4822, 0.4465], device='cuda:0')

In [12]:
imgs.reshape(3, -1).std(dim=1)

tensor([0.2470, 0.2435, 0.2616], device='cuda:0')

In [15]:
transformed_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2470, 0.2435, 0.2616))
    ]))